In [1]:
!pwd

/home/nicksung/Desktop/nicksung/bwb_full_v2


In [2]:
## remove the beta as input and cf_y as output
## expand the size of the mlp

In [3]:
import os
import torch
from torch.utils.data import DataLoader
from dataset_v2 import UnifiedDesignDataset, design_collate_fn, split_designs
from film_model_v1 import FiLMNet
# from film_model_v2 import FiLMNet
from train import train_model




In [4]:
def main():

    # File paths (update if necessary)
    trainval_csv = "./processed/final_data_train_val.csv"
    trainval_points_hdf5 = "/home/nicksung/Desktop/nicksung/bwb_full/surface_point/surface_data_train_val.hdf5"
    trainval_normal_hdf5 = "/home/nicksung/Desktop/nicksung/bwb_full/surface_point/normals_data_train_val.hdf5"

    best_model_path = "./film_model_saved_weights/best_model_v1.pth"
    final_model_path = "./film_model_saved_weights/final_model_v1.pth"

    device = "cuda:0" if torch.cuda.is_available() else "cpu"

    # Create the dataset at the design level
    trainval_dataset = UnifiedDesignDataset(
        csv_path=trainval_csv,
        hdf5_points_path=trainval_points_hdf5,
        hdf5_normals_path=trainval_normal_hdf5, 
        mode="train"
    )

    print("Number of designs found:", len(trainval_dataset))

    # Split into train and validation sets
    train_ds, val_ds = split_designs(trainval_dataset, train_ratio=0.9)
    print("Train designs:", len(train_ds), "Val designs:", len(val_ds))

    # Build DataLoaders (each batch contains 64 designs)
    train_loader = DataLoader(train_ds, batch_size=64, shuffle=True, collate_fn=design_collate_fn)
    val_loader   = DataLoader(val_ds, batch_size=64, shuffle=False, collate_fn=design_collate_fn)


    # Create the FiLM model
    model = FiLMNet(cond_dim=13, coord_dim=6, output_dim=3, hidden_dim=256, num_layers=4, extra_layers=3)

    # Train the model with logging, saving both best and final models.
    model = train_model(train_loader, val_loader, model, epochs=20000, lr=5e-4, device=device,
                        log_file="training_log.txt", best_model_path=best_model_path, final_model_path=final_model_path)


if __name__ == '__main__':
    main()

Unique geometries in CSV: 999
Unique geometries in dataset after HDF5 filtering: 999
Number of designs found: 999
Train designs: 899 Val designs: 100
Epoch 1/20000 | Train MSE: 0.975629 (cp: 0.951478, cf_x: 0.993065, cf_z: 0.982345) | Val MSE: 0.682182 (cp: 0.964173, cf_x: 0.551298, cf_z: 0.531076)
Epoch 100/20000 | Train MSE: 0.303203 (cp: 0.140352, cf_x: 0.244925, cf_z: 0.524334) | Val MSE: 0.351280 (cp: 0.116460, cf_x: 0.315573, cf_z: 0.621808)
Epoch 200/20000 | Train MSE: 0.184616 (cp: 0.074721, cf_x: 0.128326, cf_z: 0.350802) | Val MSE: 0.153204 (cp: 0.070057, cf_x: 0.105261, cf_z: 0.284294)
Epoch 300/20000 | Train MSE: 0.261615 (cp: 0.104137, cf_x: 0.253903, cf_z: 0.426805) | Val MSE: 0.376019 (cp: 0.143838, cf_x: 0.475540, cf_z: 0.508680)
Epoch 400/20000 | Train MSE: 0.184561 (cp: 0.063135, cf_x: 0.123753, cf_z: 0.366794) | Val MSE: 0.199421 (cp: 0.072200, cf_x: 0.153031, cf_z: 0.373032)
Epoch 500/20000 | Train MSE: 0.138166 (cp: 0.048952, cf_x: 0.104962, cf_z: 0.260584) | Val M

In [5]:
from film_model_v1 import FiLMNet
model = FiLMNet(cond_dim=13, coord_dim=6, output_dim=3, hidden_dim=256, num_layers=4, extra_layers=3)

def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)

print("Total trainable parameters:", count_parameters(model))


Total trainable parameters: 795651
